In [42]:
#-----------------------------------------pre-processing---------------------------------------------------#

#data loading
import pandas as pd
predata = pd.read_csv('traindata.csv', nrows = 1000) #aumentar nrows até rebentar

In [2]:
#len(predata[predata['clientid'] == 3])  #3, 194, 259

In [44]:
#predata['clientid'].value_counts()

194    140
259    104
3       60
361     35
19      27
2       26
154     26
113     23
280     20
421     19
377     17
248     17
233     16
119     15
266     15
181     14
197     14
297     13
191     13
184     12
425     12
116     12
431     11
75      11
220     10
202     10
16      10
410     10
216      9
36       9
353      9
134      9
300      9
404      8
239      8
277      8
213      8
27       8
110      8
175      7
178      7
336      7
308      7
321      6
106      6
78       6
270      6
130      6
313      5
54       5
368      5
67       5
50       5
325      5
293      5
103      5
229      5
448      4
440      4
142      4
205      4
44       4
47       4
400      4
383      4
58       4
209      3
137      3
372      3
62       3
86       3
40       3
428      3
96       3
367      2
356      2
349      2
122      2
289      2
285      2
236      2
82       2
89       2
99       2
330      1
263      1
252      1
163      1
31       1
407      1
417      1

In [3]:
predata_by_id = predata[predata['clientid'] == 3][['timestamp', 'canal', 'operativa', 'entity', 'reference', 
                                                   'trusted_indicator', 'iban_orig', 'iban_dest', 'amount', 'accountbalance',
                                                  'ipaddress', 'is_fraud', 'is_mobile', 'is_tablet', 'is_pc', 'is_touch',
                                                  'is_bot', 'browser_family', 'os_family', '#clientid_30D', '#ipaddress_30D',
                                                '#iban_orig_30D', '#iban_dest_30D', 'clientid_time_diff', 'iban_dest_time_diff',
                                                'mean_amount_clientid_30D', 'mean_amount_iban_dest_30D', '#clientid_ipaddress_30D', 
                                                '#clientid_iban_orig_30D', '#clientid_iban_dest_30D', 'is_fraud_cons', 'date', 
                                                'hour', 'cons_freq_ipaddress']]

In [4]:
#columns created by me
predata_by_id['weekday'] = pd.to_datetime(predata_by_id['timestamp']).apply(lambda x: x.weekday())

In [5]:
predata_by_id.drop(['timestamp'], axis = 1, inplace = True)

In [6]:
#remove dots from ipaddress
predata_by_id['ipaddress'] = predata_by_id['ipaddress'].apply(lambda x: "".join(x.split(".")))

In [7]:
#convert decimals to int 
cols_to_int = ['#clientid_30D', '#ipaddress_30D', '#iban_orig_30D', '#iban_dest_30D', 'clientid_time_diff',
               'iban_dest_time_diff', '#clientid_ipaddress_30D', '#clientid_iban_orig_30D', '#clientid_iban_dest_30D']

for col in cols_to_int:
    predata_by_id[col] = predata_by_id[col].apply(lambda x: int(x))

In [8]:
#add letter before number to distinguish between similar words from different columns
cols = ['entity', 'reference', 'iban_orig', 'iban_dest', 'ipaddress', '#clientid_30D', '#ipaddress_30D', '#iban_orig_30D', 
        '#iban_dest_30D', 'clientid_time_diff', 'iban_dest_time_diff', '#clientid_ipaddress_30D', '#clientid_iban_orig_30D', 
        '#clientid_iban_dest_30D', 'is_fraud_cons', 'date', 'hour', 'cons_freq_ipaddress']

identifier = ['e', 'r', 'io', 'id', 'ip', 'h', 'fc', 'd', 'cfi', 'ncd', 'nid', 'niod', 'nidd', 'ctd', 'idtd', 'ncid', 
'nciod', 'ncidd']

for col in range(len(cols)):
    predata_by_id[cols[col]] = predata_by_id[cols[col]].apply(lambda x: identifier[col] + str(x))

In [9]:
#convert binary and UNKNOWN to words 
def apply_map(df, col, target_map):
    df[col] = df[col].apply(lambda x: target_map.get(str(x)))

cols_maps = [('is_fraud', {'0':'nfraud', '1':'fraud'}),
             ('trusted_indicator', {'0.0':'ntrusted', '1.0':'trusted', 'UNKNOWN':'unknown'}),
             ('is_mobile', {'False':'nmobile', 'True':'mobile'}),
             ('is_tablet', {'False':'ntablet', 'True':'tablet'}),
             ('is_pc', {'False':'npc', 'True':'pc'}),
             ('is_touch', {'False':'ntouch', 'True':'touch'}),
             ('is_bot', {'False':'nbot', 'True':'bot'}),
             ('weekday', {'0':'seg', '1':'ter', '2':'qua', '3':'qui', '4':'sex', '5':'sab', '6':'dom'})]

for comb in cols_maps:
    apply_map(predata_by_id, comb[0], comb[1])

In [10]:
#len(predata_by_id[predata_by_id['is_fraud'] == 'fraud'])

In [11]:
#len(predata_by_id[predata_by_id['is_fraud'] == 'nfraud'])

In [12]:
#amounts -> classes
import numpy as np 

#automatic labels
import string

class LabelCategorizer():
    def __init__(self, base_word = 'cat'):
        self._alphabet_index = 0
        self.base_word = base_word
        self.current_word = self.base_word
        self.shift = 0

    def __str__(self):
        return 'Class: Label Categorizer\nBase word: ' + self.base_word + '\nCurrent Word: ' + self.current_word

    def get_next_word(self):
        if self.shift > 0:
            self.current_word = self.current_word[-1] + self.current_word[:-1]
            self.shift -= 1
        else:
            self.current_word = self.current_word + string.ascii_lowercase[self._alphabet_index]
            self._alphabet_index = (self._alphabet_index + 1) % len(string.ascii_lowercase)
            self.shift = len(self.current_word) - 1

        return self.current_word


#replacement of the old columns with the new ones with classes 
def cutter (col, number, word, words_map):

    bins_a = np.linspace(predata_by_id[col].min(), predata_by_id[col].max(), num = number)
    bins_a[0] = bins_a[0]-1
    bins_aux = bins_a[1:]
    bins_aux = np.append(bins_aux, bins_a[-1]+1)
    bin_tuples = list(zip(bins_a, bins_aux))

    bins = pd.IntervalIndex.from_tuples(bin_tuples)

    labels_a = []

    a = LabelCategorizer(base_word = word)

    for _ in range(number):
        labels_a.append(a.get_next_word())

    x = pd.cut(predata_by_id[col].to_list(), bins = bins)
    x.categories = labels_a
    predata_by_id[col] = x

    
    for i in range(number):
        words_map[labels_a[i]] = bins[i]       

        
columns = ['amount', 'accountbalance', 'mean_amount_clientid_30D', 'mean_amount_iban_dest_30D']
number_bins = [100, 250, 100, 100]
base_words = ['blue', 'pink', 'red', 'grey']

values_map = {}

for i in range (len(columns)):
    cutter(columns[i], number_bins[i], base_words[i], values_map)

In [13]:
#values_map

In [14]:
#values_map['pinka']

In [15]:
predata_by_id['canal'] = predata_by_id['canal'].str.lower()
predata_by_id['operativa'] = predata_by_id['operativa'].str.lower()
predata_by_id['browser_family'] = predata_by_id['browser_family'].str.lower()
predata_by_id['os_family'] = predata_by_id['os_family'].str.lower()
predata_by_id['ipaddress'] = predata_by_id['ipaddress'].str.lower()

In [36]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#predata_by_id

In [17]:
#dataframe to numpy array of arrays
sentences = predata_by_id.to_numpy()

In [18]:
sentences[0]

array(['obe', 'trfsep', 'e3', 'r3', 'ntrusted', 'io2935722', 'id1110992',
       'lueab', 'kapin', 'ip959216914', 'nfraud', 'nmobile', 'ntablet',
       'npc', 'ntouch', 'nbot', 'other', 'other', 'h0', 'fc14', 'd1',
       'cfi6', 'ncd-1', 'nid1113', 'edarb', 'yeagbrced', 'niod0', 'nidd0',
       'ctd0', 'idtd0', 'ncid20200221', 'nciod20', 'ncidd1', 'sex'],
      dtype=object)

In [19]:
#np.savetxt('train_data.txt', sentences, delimiter=" ", fmt="%s")
#sentences = w2v.LineSentence('train_data.txt')

In [20]:
sentences_aux = [list(curr) for curr in sentences]

In [21]:
sentences_aux[0]

['obe',
 'trfsep',
 'e3',
 'r3',
 'ntrusted',
 'io2935722',
 'id1110992',
 'lueab',
 'kapin',
 'ip959216914',
 'nfraud',
 'nmobile',
 'ntablet',
 'npc',
 'ntouch',
 'nbot',
 'other',
 'other',
 'h0',
 'fc14',
 'd1',
 'cfi6',
 'ncd-1',
 'nid1113',
 'edarb',
 'yeagbrced',
 'niod0',
 'nidd0',
 'ctd0',
 'idtd0',
 'ncid20200221',
 'nciod20',
 'ncidd1',
 'sex']

In [22]:
sentences_series = pd.Series(sentences_aux)

In [23]:
#---------------------------------------------------training------------------------------------------------------#

#inicialization and training word2vec
import multiprocessing
import gensim.models.word2vec as w2v

def training (sentences, cycles, dim, architecture, context):
    model = w2v.Word2Vec (
        sg = architecture, #1 - skip-gram , 0 - cbow
        workers = multiprocessing.cpu_count(), #uses all the cores 
        vector_size = dim, #dimension of the embedding space = N 
        window = context, #words befores and after center word
        sample = 0, #whithout downsampling 
        ns_exponent = 0.0,
        min_count = 0, 
)
    
    #vocabulary creation
    model.build_vocab(sentences) 

    #model training
    model.train(sentences, epochs = cycles, total_examples = model.corpus_count, compute_loss = True)

    return model

#model creation
model = training(sentences_series, 5, 10, 1, 5)

In [24]:
print(model)

Word2Vec(vocab=502, vector_size=10, alpha=0.025)


In [25]:
print(len(model.wv.index_to_key)) #vocab size = V

502


In [37]:
#print(model.wv.index_to_key)

In [27]:
model.wv.most_similar('nfraud')

[('nid-1', 0.9979271292686462),
 ('nmobile', 0.9953253865242004),
 ('ntablet', 0.9933708906173706),
 ('npc', 0.9932478666305542),
 ('fc39', 0.9921668171882629),
 ('h21', 0.9921357035636902),
 ('niod21', 0.9906290173530579),
 ('d4', 0.9895514845848083),
 ('d18', 0.9894483089447021),
 ('fc3', 0.9884853363037109)]

In [28]:
#saving the model
model.save('Trained_by_id.w2v')

In [29]:
#-------------------------------------------------matrices--------------------------------------------------------#

#weight matrices
m1 = model.wv.vectors  #input embedding matrix  #VxN

In [30]:
m2 = model.syn1neg      #NxV 

In [31]:
#visualization of the matrices

#np.set_printoptions(edgeitems = 10000)
#np.set_printoptions(linewidth = 10000)
np.set_printoptions(threshold = np.inf)

#import sys
#np.set_printoptions(threshold = sys.maxsize)

In [38]:
print(m1.shape, m2.shape)
import numpy as np
#print(np.matrix(m1))
#print()
#print()
#print(np.matrix(m2))

(502, 10) (502, 10)


In [33]:
m3 = np.transpose(m2)
#print(m3.shape)
#print(np.matrix(m3))

In [34]:
#condicional probabilities
probabilities = np.matmul(m1, m3)

print(probabilities.shape)

(502, 502)


In [40]:
#seeing the matrix
#s = [[str(e) for e in row] for row in probabilities]
#lens = [max(map(len, col)) for col in zip(*s)]
#fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
#table = [fmt.format(*row) for row in s]
#print ('\n'.join(table))

#print(np.matrix(probabilities))
np.savetxt('prob_by_id.txt', probabilities)